In [1]:
!pip install gdown ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.2 MB/s eta 0:00:00


In [ ]:
!gdown https://drive.google.com/file/d/1brZzNDWhryZHaVAsHHtRzQRekq3uRQ5z/view?usp=sharing --fuzzy # this da fred-0 1
!unzip yolo_fred-0.zip

In [3]:
from pathlib import Path
import random
import shutil
import yaml

# ======================
# 1. Source paths
# ======================

IMAGES_SRC = Path("/content/yolo_frames")
LABELS_SRC = Path("/content/yolo_annotated")

DATASET = Path("/content/yolo_dataset")
TRAIN_IMG = DATASET / "images/train"
VAL_IMG   = DATASET / "images/val"
TRAIN_LAB = DATASET / "labels/train"
VAL_LAB   = DATASET / "labels/val"

for p in [TRAIN_IMG, VAL_IMG, TRAIN_LAB, VAL_LAB]:
    p.mkdir(parents=True, exist_ok=True)

# ======================
# 2. Match img/txt pairs
# ======================

images = sorted([p for p in IMAGES_SRC.iterdir() if p.suffix.lower() in [".jpg", ".png", ".jpeg"]])

pairs = []
for img in images:
    txt = LABELS_SRC / (img.stem + ".txt")
    if txt.exists():
        pairs.append((img, txt))

print("Valid pairs:", len(pairs))

# split
random.seed(42)
random.shuffle(pairs)
split = int(0.8 * len(pairs))
train_pairs = pairs[:split]
val_pairs = pairs[split:]

def copy_pairs(pairs, img_dst, lab_dst):
    for img, lab in pairs:
        shutil.copy2(img, img_dst / img.name)
        shutil.copy2(lab, lab_dst / lab.name)

copy_pairs(train_pairs, TRAIN_IMG, TRAIN_LAB)
copy_pairs(val_pairs,   VAL_IMG,   VAL_LAB)

print("Dataset ready at:", DATASET)

# ======================
# 3. Write data.yaml
# ======================

# Adjust nc + names to match your dataset
data_yaml = {
    "path": str(DATASET),
    "train": "images/train",
    "val": "images/val",
    "nc": 1,
    "names": ["object"]
}

DATA_YAML = DATASET / "data.yaml"
with open(DATA_YAML, "w") as f:
    yaml.dump(data_yaml, f, sort_keys=False)

print("Created:", DATA_YAML)

# ======================
# 4. Train YOLOv8 (Ultralytics)
# ======================

from ultralytics import YOLO

# Load pretrained model
model = YOLO("yolov8n.pt")

# Single most idle GPU
results = model.train(
    data=str(DATA_YAML),
    epochs=100,
    imgsz=640,
    device=-1
)

# OR: two most idle GPUs (your example)
# results = model.train(
#     data=str(DATA_YAML),
#     epochs=100,
#     imgsz=640,
#     device=[-1, -1]
# )


Valid pairs: 3348
Dataset ready at: /content/yolo_dataset
Created: /content/yolo_dataset/data.yaml
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Searching for 1 idle GPUs with free memory >= 20.0% and free utilization >= 0.0%...
Selected idle CUDA devices [0]
Ultralytics 8.3.228 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/yolo_dataset/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=Fa

KeyboardInterrupt: 

In [8]:
import random
from pathlib import Path
from ultralytics import YOLO
import imageio.v2 as imageio

# =======================================
# 1. Setup
# =======================================

FRAMES_DIR = Path("/content/yolo_frames")
MODEL_PATH = "/content/runs/detect/train/weights/best.pt"
GIF_PATH = "/content/predicted_sequence.gif"

FPS = 30
DURATION = 10
NUM_FRAMES = FPS * DURATION  # 300 frames

model = YOLO(MODEL_PATH)

# =======================================
# 2. Collect frames
# =======================================

frame_list = sorted([
    p for p in FRAMES_DIR.iterdir()
    if p.suffix.lower() in [".jpg", ".jpeg", ".png"]
])

if len(frame_list) < NUM_FRAMES:
    raise ValueError("Not enough frames for a 10s sequence.")

start = random.randint(0, len(frame_list) - NUM_FRAMES)
sequence = frame_list[start:start + NUM_FRAMES]

print(f"Using frames {start} → {start + NUM_FRAMES - 1}")

# =======================================
# 3. Run YOLO on the sequence
# =======================================

results = model.predict(
    source=[str(f) for f in sequence],
    save=True,
    conf=0.7
)

# YOLO returns the exact output directory here:
pred_dir = Path(results[0].save_dir)
print("Predictions saved in:", pred_dir)

# =======================================
# 4. Build animated GIF
# =======================================

predicted_frames = sorted([
    p for p in pred_dir.iterdir()
    if p.suffix.lower() in [".jpg", ".jpeg", ".png"]
])

if len(predicted_frames) < 2:
    raise RuntimeError("Only one predicted frame found — GIF cannot animate.")

images = [imageio.imread(p) for p in predicted_frames]

# duration controls speed: seconds per frame
imageio.mimsave(GIF_PATH, images, duration=1/FPS)

print("Animated GIF saved at:", GIF_PATH)


Using frames 572 → 871

0: 384x640 (no detections), 2.2ms
1: 384x640 (no detections), 2.2ms
2: 384x640 (no detections), 2.2ms
3: 384x640 (no detections), 2.2ms
4: 384x640 (no detections), 2.2ms
5: 384x640 (no detections), 2.2ms
6: 384x640 (no detections), 2.2ms
7: 384x640 (no detections), 2.2ms
8: 384x640 (no detections), 2.2ms
9: 384x640 (no detections), 2.2ms
10: 384x640 (no detections), 2.2ms
11: 384x640 (no detections), 2.2ms
12: 384x640 (no detections), 2.2ms
13: 384x640 1 object, 2.2ms
14: 384x640 1 object, 2.2ms
15: 384x640 (no detections), 2.2ms
16: 384x640 1 object, 2.2ms
17: 384x640 1 object, 2.2ms
18: 384x640 (no detections), 2.2ms
19: 384x640 1 object, 2.2ms
20: 384x640 1 object, 2.2ms
21: 384x640 (no detections), 2.2ms
22: 384x640 (no detections), 2.2ms
23: 384x640 1 object, 2.2ms
24: 384x640 1 object, 2.2ms
25: 384x640 1 object, 2.2ms
26: 384x640 1 object, 2.2ms
27: 384x640 1 object, 2.2ms
28: 384x640 1 object, 2.2ms
29: 384x640 (no detections), 2.2ms
30: 384x640 1 object